OpenAI´s function_Calling was first deprecated in favor of tool_calling - but now the correct terms seems to be function_calling again.
If you call it tool- or function calling - at the moment is does not really matter.

Reference: https://platform.openai.com/docs/guides/function-calling


In [1]:
from dotenv import load_dotenv, find_dotenv
import os

load_dotenv(find_dotenv())

True

In [2]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_deployment="gpt-4o",
    openai_api_version="2024-08-01-preview",
)

In [3]:
llm.invoke("How will the weather be in munich today?")

Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized: Using outdated v1 api key. Please use v2 api key."}\n')trace=1d6e0a2f-4691-4b4f-9cae-c988821a6858,id=1d6e0a2f-4691-4b4f-9cae-c988821a6858


AIMessage(content="I'm sorry, but I cannot provide real-time weather updates. For up-to-date weather information in Munich, you can check reliable weather services such as [Weather.com](https://www.weather.com), [AccuWeather](https://www.accuweather.com), or a local weather website for Munich.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 17, 'total_tokens': 77, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b705f0c291', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'viol

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized: Using outdated v1 api key. Please use v2 api key."}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized: Using outdated v1 api key. Please use v2 api key."}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized: Using outdated v1 api key. Please use v2 api key."}

In [4]:
from langchain_core.tools import tool


@tool
def fake_weather_api(city: str) -> str:
    """
    Check the weather in a specified city.

    Args:
        city (str): The name of the city where you want to check the weather.

    Returns:
        str: A description of the current weather in the specified city.
    """
    return "Sunny, 22°C"


@tool
def outdoor_seating_availability(city: str) -> str:
    """
    Check if outdoor seating is available at a specified restaurant in a given city.

    Args:
        city (str): The name of the city where you want to check for outdoor seating availability.

    Returns:
        str: A message stating whether outdoor seating is available or not.
    """
    return "Outdoor seating is available."


tools = [fake_weather_api, outdoor_seating_availability]

In [ ]:
# from langchain_core.pydantic_v1 import BaseModel, Field

# class WeatherCheck(BaseModel):
#     """Check the weather in a specified city."""

#     city: str = Field(..., description="Name of the city to check the weather for")


# class OutdoorSeatingCheck(BaseModel):
#     """Check if outdoor seating is available at a specified restaurant in a given city."""

#     city: str = Field(..., description="Name of the city where the restaurant is located")


# tools = [WeatherCheck, OutdoorSeatingCheck]

In [5]:
llm_with_tools = llm.bind_tools(tools)

In [6]:
result = llm_with_tools.invoke("How will the weather be in munich today?")
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_D5V3e8JsuAG9m8SUBo4poNiv', 'function': {'arguments': '{"city":"munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 158, 'total_tokens': 174, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b705f0c291', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_resu

In [7]:
result = llm_with_tools.invoke(
    "How will the weather be in munich today? Do you still have seats outdoor available?"
)
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8Cb1m4VuOEoXpwQ6ffAHzHwe', 'function': {'arguments': '{"city": "Munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}, {'id': 'call_42tTejFGVsOJoVrFbtz7CW0Z', 'function': {'arguments': '{"city": "Munich"}', 'name': 'outdoor_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 166, 'total_tokens': 217, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b705f0c291', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': Fa

In [8]:
result.tool_calls

[{'name': 'fake_weather_api',
  'args': {'city': 'Munich'},
  'id': 'call_8Cb1m4VuOEoXpwQ6ffAHzHwe',
  'type': 'tool_call'},
 {'name': 'outdoor_seating_availability',
  'args': {'city': 'Munich'},
  'id': 'call_42tTejFGVsOJoVrFbtz7CW0Z',
  'type': 'tool_call'}]

In [9]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [
    HumanMessage(
        "How will the weather be in munich today? I would like to eat outside if possible"
    )
]
llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)

In [10]:
messages

[HumanMessage(content='How will the weather be in munich today? I would like to eat outside if possible', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_BsJcv9n1eigv4dhroxWQ8hCX', 'function': {'arguments': '{"city": "munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}, {'id': 'call_MeS2TBROJj0e3ENAbWXfNrLY', 'function': {'arguments': '{"city": "munich"}', 'name': 'outdoor_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 166, 'total_tokens': 217, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b705f0c291', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, '

In [11]:
tool_mapping = {
    "fake_weather_api": fake_weather_api,
    "outdoor_seating_availability": outdoor_seating_availability,
}

In [12]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output = tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [13]:
messages

[HumanMessage(content='How will the weather be in munich today? I would like to eat outside if possible', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_BsJcv9n1eigv4dhroxWQ8hCX', 'function': {'arguments': '{"city": "munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}, {'id': 'call_MeS2TBROJj0e3ENAbWXfNrLY', 'function': {'arguments': '{"city": "munich"}', 'name': 'outdoor_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 166, 'total_tokens': 217, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b705f0c291', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, '

In [14]:
llm_with_tools.invoke(messages)

AIMessage(content='The weather in Munich today is sunny with a temperature of 22°C. Good news—outdoor seating is available, so you can enjoy eating outside!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 245, 'total_tokens': 277, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b705f0c291', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filte